# Data Cleansing - Monthly Retail and Food Services Sales from 1992 to 2020

## Data import and merge into 1 DataFrame

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import math

# Set DataFrame display options to avoid truncating long data
pd.set_option('max_colwidth', 1000) # Show up to 1000 characters within each cell
pd.set_option('max_rows', 20) # Show up to 20 dataframe rows
pd.set_option('max_columns', 1000) # Show up to 1000 columns

# Managing Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import the workbook containing data from 1992 to 2019 and append sheet name as a separate column
def read_excel_sheets(xls_path):
    # Read all sheets of an Excel workbook and return a single DataFrame
    print(f'Loading {xls_path} into pandas')
    xl = pd.ExcelFile(xls_path)
    df = pd.DataFrame()
    columns = None
    for idx, name in enumerate(xl.sheet_names):
        print(f'Reading sheet #{idx}: {name}')
        sheet = xl.parse(name)
        if idx == 0:
            # Save column names from the first sheet to match for append
            columns = sheet.columns
        sheet.columns = columns
        # Add sheet name as column
        sheet['sheet'] = name.split(" ")[-1]
        # Assume index of existing data frame when appended
        df = df.append(sheet, ignore_index = True)
    return df

In [3]:
data = read_excel_sheets('C:/Users/Lenovo/Downloads/DataCleansing/mrtssales92-2019.xls')

Loading C:/Users/Lenovo/Downloads/DataCleansing/mrtssales92-2019.xls into pandas
Reading sheet #0: 2019
Reading sheet #1: 2018
Reading sheet #2: 2017
Reading sheet #3: 2016
Reading sheet #4: 2015
Reading sheet #5: 2014
Reading sheet #6: 2013
Reading sheet #7: 2012
Reading sheet #8: 2011
Reading sheet #9: 2010
Reading sheet #10: 2009
Reading sheet #11: 2008
Reading sheet #12: 2007
Reading sheet #13: 2006
Reading sheet #14: 2005
Reading sheet #15: 2004
Reading sheet #16: 2003
Reading sheet #17: 2002
Reading sheet #18: 2001
Reading sheet #19: 2000
Reading sheet #20: 1999
Reading sheet #21: 1998
Reading sheet #22: 1997
Reading sheet #23: 1996
Reading sheet #24: 1995
Reading sheet #25: 1994
Reading sheet #26: 1993
Reading sheet #27: 1992


In [4]:
# Rename columns to standardised names
data = data.rename(columns = {'Estimates of Monthly Retail and Food Services Sales by Kind of Business: 2019':'NAICS Code',
                'Unnamed: 1':'Kind of Business', 'Unnamed: 2':'January', 'Unnamed: 3':'February', 'Unnamed: 4':'March', 'Unnamed: 5':'April',
                'Unnamed: 6':'May', 'Unnamed: 7':'June', 'Unnamed: 8':'July', 'Unnamed: 9':'August', 'Unnamed: 10': 'September',
                'Unnamed: 11':'October', 'Unnamed: 12':'November', 'Unnamed: 13':'December', 'Unnamed: 14':'Total', 'sheet':'Year'})
data = data.drop(columns = 'Total')
data.shape

(3247, 15)

In [5]:
# Data for 2020 has a slight different tabular format and will be processed separately before appending with other data
df_2020 = pd.read_excel('C:/Users/Lenovo/Downloads/DataCleansing/mrtssales2020.xls', sheet_name = '2020')
df_2020.columns

Index(['Estimates of Monthly Retail and Food Services Sales by Kind of Business: 2020',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')

In [6]:
# Rename columns to standardised names
df_2020 = df_2020.rename(columns = {'Estimates of Monthly Retail and Food Services Sales by Kind of Business: 2020':'NAICS Code',
                'Unnamed: 1':'Kind of Business', 'Unnamed: 2':'January', 'Unnamed: 3':'February', 'Unnamed: 4':'March', 'Unnamed: 5':'April',
                'Unnamed: 6':'May', 'Unnamed: 7':'June', 'Unnamed: 8':'July', 'Unnamed: 9':'August', 'Unnamed: 10': 'September',
                'Unnamed: 11':'CY CUM', 'Unnamed: 12':'PY CUM'})

# Align tabular structure to be consistent with data from 1992 to 2019 
df_2020 = df_2020.drop(columns =['CY CUM','PY CUM'])
df_2020['Year'] = '2020'

df_2020.shape

(120, 12)

In [7]:
# Append both dataset together
data = pd.concat([data, df_2020], ignore_index = True, sort = False)

In [8]:
data.shape

(3367, 15)

## Remove redundant rows & enrich categorical data

In [9]:
data = data[data['Kind of Business'].notna()]
data = data[data['Kind of Business'] != 'Kind of Business']
data.head(4000)

,NAICS Code,Kind of Business,January,February,March,April,May,June,July,August,September,October,November,December,Year
4,NaN,NOT ADJUSTED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
5,NaN,"Retail and food services sales, total",458089,443708,515694,509413,547130,518273,532103,545247,496074,525539,535352,591380,2019
6,NaN,Retail sales and food services excl motor vehicle and parts,370975,354699,405849,406257,435358,413530,422834,430588,396537,421738,434251,487917,2019
7,NaN,Retail sales and food services excl gasoline stations,422675,409159,474734,466269,500912,474450,486653,500204,454138,481937,494622,551177,2019
8,NaN,Retail sales and food services excl motor vehicle and parts and gasoline stations,335561,320150,364889,363113,389140,369707,377384,385545,354601,378136,393521,447714,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3351,453,Miscellaneous stores retailers,11984,11877,10230,7591,9091,11088,11548,11712,11836,NaN,NaN,NaN,2020
3352,454,Nonstore retailers,68101,68924,72365,79143,85256,82900,83201,87008,85508,NaN,NaN,NaN,2020
3353,4541,Electronic shopping and mail order houses,60985,61620,65332,73479,79351,76398,75767,79452,77884,NaN,NaN,NaN,2020
3354,45431,Fuel dealers,2096,2107,1974,1937,2017,2009,2028,2038,2098,NaN,NaN,NaN,2020


In [10]:
data = data.reset_index(drop=True)

In [11]:
# Add a column to differentiate between Adjusted Values and Unadjusted Values
data['Not Adjusted/ Adjusted'] = np.nan

data.loc[data['Kind of Business'] == 'NOT ADJUSTED', 'Not Adjusted/ Adjusted'] = 'Not Adjusted'
data.loc[data['Kind of Business'] == 'ADJUSTED(2)', 'Not Adjusted/ Adjusted'] = 'Adjusted'
data['Not Adjusted/ Adjusted'] = data['Not Adjusted/ Adjusted'].fillna(method = 'ffill')

data.head(50)

,NAICS Code,Kind of Business,January,February,March,April,May,June,July,August,September,October,November,December,Year,Not Adjusted/ Adjusted
0,NaN,NOT ADJUSTED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,Not Adjusted
1,NaN,"Retail and food services sales, total",458089,443708,515694,509413,547130,518273,532103,545247,496074,525539,535352,591380,2019,Not Adjusted
2,NaN,Retail sales and food services excl motor vehicle and parts,370975,354699,405849,406257,435358,413530,422834,430588,396537,421738,434251,487917,2019,Not Adjusted
3,NaN,Retail sales and food services excl gasoline stations,422675,409159,474734,466269,500912,474450,486653,500204,454138,481937,494622,551177,2019,Not Adjusted
4,NaN,Retail sales and food services excl motor vehicle and parts and gasoline stations,335561,320150,364889,363113,389140,369707,377384,385545,354601,378136,393521,447714,2019,Not Adjusted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,451211,Book stores,1004,569,587,631,675,594,572,1088,842,627,618,1037,2019,Not Adjusted
46,452,General merchandise stores,51557,50513,58619,56815,60695,58727,58487,61460,54515,58560,66027,77648,2019,Not Adjusted
47,4521,Department stores,8657,9142,10645,10456,11299,10625,10630,11550,9762,10456,13401,18403,2019,Not Adjusted
48,452111,Department stores(excl. discount department stores),2464,2803,3263,3285,3488,3199,3130,3253,3045,3109,4390,6561,2019,Not Adjusted


In [12]:
# Remove empty rows without any monthly value
data = data[data['January'].notna()]
data.head(100)

,NAICS Code,Kind of Business,January,February,March,April,May,June,July,August,September,October,November,December,Year,Not Adjusted/ Adjusted
1,NaN,"Retail and food services sales, total",458089,443708,515694,509413,547130,518273,532103,545247,496074,525539,535352,591380,2019,Not Adjusted
2,NaN,Retail sales and food services excl motor vehicle and parts,370975,354699,405849,406257,435358,413530,422834,430588,396537,421738,434251,487917,2019,Not Adjusted
3,NaN,Retail sales and food services excl gasoline stations,422675,409159,474734,466269,500912,474450,486653,500204,454138,481937,494622,551177,2019,Not Adjusted
4,NaN,Retail sales and food services excl motor vehicle and parts and gasoline stations,335561,320150,364889,363113,389140,369707,377384,385545,354601,378136,393521,447714,2019,Not Adjusted
5,NaN,"Retail sales, total",401341,386840,449507,445959,479466,452242,466160,477537,433299,460534,473026,526295,2019,Not Adjusted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,4529,Other general merchandise stores,47638,47104,48029,48225,48090,48268,48516,48422,48285,48620,48509,48641,2019,Adjusted
98,45291,Warehouse clubs and superstores,40775,40387,41159,41282,41179,41364,41596,41520,41433,41760,41635,41823,2019,Adjusted
99,45299,All other gen. merchandise stores,6863,6717,6870,6943,6911,6904,6920,6902,6852,6860,6874,6818,2019,Adjusted
100,453,Miscellaneous stores retailers,10616,10650,10825,11083,11159,11269,11224,11459,11449,11533,11652,11485,2019,Adjusted


In [13]:
# Remove subtotal rows without any NAICS Code
data = data[data['NAICS Code'].notna()]
data.head(100)

,NAICS Code,Kind of Business,January,February,March,April,May,June,July,August,September,October,November,December,Year,Not Adjusted/ Adjusted
8,441,Motor vehicle and parts dealers,87114,89009,109845,103156,111772,104743,109269,114659,99537,103801,101101,103463,2019,Not Adjusted
9,"4411,4412",Automobile and other motor vehicle dealers,80026,82022,101593,95035,103357,96728,100916,106073,91783,95254,93242,96073,2019,Not Adjusted
10,4411,Automobile dealers,75271,76634,93879,86774,94030,87931,92447,98149,85390,89148,87851,91114,2019,Not Adjusted
11,44111,New car dealers,66443,66959,82023,76429,83481,77853,82192,87792,75986,79452,78496,82445,2019,Not Adjusted
12,44112,Used car dealers,8828,9675,11856,10345,10549,10078,10255,10357,9404,9696,9355,8669,2019,Not Adjusted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,"442,443","Furniture, home furn, electronics, and appliance stores",16467,15935,18084,16467,17799,17713,17563,18545,17312,17548,20868,22799,2018,Not Adjusted
120,442,Furniture and home furnishings stores,8686,8504,10016,9205,9949,9751,9740,10209,9573,9758,10579,10925,2018,Not Adjusted
121,4421,Furniture stores,4597,4677,5524,4890,5254,5269,5159,5396,5225,5027,5314,5229,2018,Not Adjusted
122,4422,Home furnishings stores,4089,3827,4492,4315,4695,4482,4581,4813,4348,4731,5265,5696,2018,Not Adjusted


Upon analysing the dataset, we can observe that there are many level of aggregation based on the North American Industry Classification System (NAICS). In the below section, we will add some flag to classify the level of aggregation to avoid double counting. We will be using the below description for our flag.
- L0: 441, 442, 443 and so on
- L1: sub-class of L0 (for example, 4411, 4413)
- L2: sub-class of L1 (for example, 44111, 44112)
- L3: sub-class of L2 (for example, 443141, 443142)
- A: for rows which include the sum of value across multiple NAICS code. These rows will be removed from the dataset to avoid double counting.

In [14]:
# Calculate length of NAICS Code column to determine the appropriate flag
data['NAICS Code Length'] = data['NAICS Code'].astype(str).str.len()
data.head()

,NAICS Code,Kind of Business,January,February,March,April,May,June,July,August,September,October,November,December,Year,Not Adjusted/ Adjusted,NAICS Code Length
8,441,Motor vehicle and parts dealers,87114,89009,109845,103156,111772,104743,109269,114659,99537,103801,101101,103463,2019,Not Adjusted,3
9,"4411,4412",Automobile and other motor vehicle dealers,80026,82022,101593,95035,103357,96728,100916,106073,91783,95254,93242,96073,2019,Not Adjusted,9
10,4411,Automobile dealers,75271,76634,93879,86774,94030,87931,92447,98149,85390,89148,87851,91114,2019,Not Adjusted,4
11,44111,New car dealers,66443,66959,82023,76429,83481,77853,82192,87792,75986,79452,78496,82445,2019,Not Adjusted,5
12,44112,Used car dealers,8828,9675,11856,10345,10549,10078,10255,10357,9404,9696,9355,8669,2019,Not Adjusted,5


In [15]:
# Implement the flag & remove rows containing sum of values across multiple NAICS code
conditions = [(data['NAICS Code Length'] == 3), 
              (data['NAICS Code Length'] == 4), 
              (data['NAICS Code Length'] == 5), 
              (data['NAICS Code Length'] == 6), 
              (data['NAICS Code Length'] > 6)]

values = ['L0', 'L1', 'L2', 'L3', 'A']

data['Flag'] = np.select(conditions, values)

data = data[data['Flag']!='A']

data.head()

,NAICS Code,Kind of Business,January,February,March,April,May,June,July,August,September,October,November,December,Year,Not Adjusted/ Adjusted,NAICS Code Length,Flag
8,441,Motor vehicle and parts dealers,87114,89009,109845,103156,111772,104743,109269,114659,99537,103801,101101,103463,2019,Not Adjusted,3,L0
10,4411,Automobile dealers,75271,76634,93879,86774,94030,87931,92447,98149,85390,89148,87851,91114,2019,Not Adjusted,4,L1
11,44111,New car dealers,66443,66959,82023,76429,83481,77853,82192,87792,75986,79452,78496,82445,2019,Not Adjusted,5,L2
12,44112,Used car dealers,8828,9675,11856,10345,10549,10078,10255,10357,9404,9696,9355,8669,2019,Not Adjusted,5,L2
13,4413,"Automotive parts, acc., and tire stores",7088,6987,8252,8121,8415,8015,8353,8586,7754,8547,7859,7390,2019,Not Adjusted,4,L1


To make it easier to perform aggregation later across different codes, we will also be adding an additional column to indicate the Parent NAICS Code. For example, the parent NAICS Code of 4411 is 441. Rows with the same Parent NAICS Code can be summed together to represent the total of the corresponding parent NAICS Code.

In [16]:
# Add the Parent NAICS Code 
data['Parent NAICS Code'] = data['NAICS Code'].astype(str).str[:-1]

data.head()

,NAICS Code,Kind of Business,January,February,March,April,May,June,July,August,September,October,November,December,Year,Not Adjusted/ Adjusted,NAICS Code Length,Flag,Parent NAICS Code
8,441,Motor vehicle and parts dealers,87114,89009,109845,103156,111772,104743,109269,114659,99537,103801,101101,103463,2019,Not Adjusted,3,L0,44
10,4411,Automobile dealers,75271,76634,93879,86774,94030,87931,92447,98149,85390,89148,87851,91114,2019,Not Adjusted,4,L1,441
11,44111,New car dealers,66443,66959,82023,76429,83481,77853,82192,87792,75986,79452,78496,82445,2019,Not Adjusted,5,L2,4411
12,44112,Used car dealers,8828,9675,11856,10345,10549,10078,10255,10357,9404,9696,9355,8669,2019,Not Adjusted,5,L2,4411
13,4413,"Automotive parts, acc., and tire stores",7088,6987,8252,8121,8415,8015,8353,8586,7754,8547,7859,7390,2019,Not Adjusted,4,L1,441


## Correct missing numerical values

In [17]:
# Replace missing numerical values (denoted with the string '(S)') with 0
data = data.replace(r'(S)', 0)

# Replace missing numerical values (denoted with the string '(NA)') with 0
data = data.replace(r'(NA)', 0)

## Unpivot and export clean data to Excel file

In [18]:
data.columns

Index(['NAICS Code', 'Kind of Business', 'January', 'February', 'March',
       'April', 'May', 'June', 'July', 'August', 'September', 'October',
       'November', 'December', 'Year', 'Not Adjusted/ Adjusted',
       'NAICS Code Length', 'Flag', 'Parent NAICS Code'],
      dtype='object')

In [19]:
# Unpivot data
data_unpivot = pd.melt(data,id_vars = ['Parent NAICS Code', 'NAICS Code', 'Flag', 'Kind of Business',
                                       'Not Adjusted/ Adjusted', 'Year'], 
                       value_vars = ['January', 'February', 'March', 'April', 'May', 'June', 
                                     'July', 'August', 'September', 'October','November', 'December'], 
                       var_name = 'Month',
                       value_name = 'Monthly Sales ($ millions)')
data_unpivot.head()

,Parent NAICS Code,NAICS Code,Flag,Kind of Business,Not Adjusted/ Adjusted,Year,Month,Monthly Sales ($ millions)
0,44,441,L0,Motor vehicle and parts dealers,Not Adjusted,2019,January,87114.0
1,441,4411,L1,Automobile dealers,Not Adjusted,2019,January,75271.0
2,4411,44111,L2,New car dealers,Not Adjusted,2019,January,66443.0
3,4411,44112,L2,Used car dealers,Not Adjusted,2019,January,8828.0
4,441,4413,L1,"Automotive parts, acc., and tire stores",Not Adjusted,2019,January,7088.0


In [20]:
# Drop rows related to year 2020 from October to December because there is no monthly sales value recorded yet
data_unpivot = data_unpivot.dropna(subset=['Monthly Sales ($ millions)'])

In [21]:
# Write to Excel file
data_unpivot.to_excel('mrtssales.xlsx', sheet_name = 'data', index = False)